# XGBoost

Lo que hace tan popular a XGBoost es la velocidad y los resultados que logra alcanzar. El algoritmos es paralelizable y por ende logra ser rápido de entrenar, se puede paralelizar en la GPU y entre una red de computadores

XGBoost ha logrado una performance al nivel del estado del arte en muchas tareas de ML

### 1- Clasificación con XGBoost

In [8]:
import xgboost as xgb
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore")

Usamos el dataset de cancer de mamas (de ML con árboles de decisión) para crear un modelo de XGBoost rápido

In [9]:
# Traemos los datos
wbc = pd.read_csv("datasets/MLTreeModels/wbc.csv")
# Reemplazamos por 1 y 0
wbc['diagnosis'] = wbc['diagnosis'].replace(['M', 'B'], [1, 0])
X = wbc[['radius_mean', 'texture_mean', 'perimeter_mean',
         'area_mean', 'smoothness_mean', 'compactness_mean',
         'concavity_mean', 'concave points_mean', 'symmetry_mean',
         'fractal_dimension_mean', 'radius_se', 'texture_se',
         'perimeter_se', 'area_se', 'smoothness_se',
         'compactness_se', 'concavity_se', 'concave points_se',
         'symmetry_se', 'fractal_dimension_se', 'radius_worst',
         'texture_worst', 'perimeter_worst', 'area_worst',
         'smoothness_worst', 'compactness_worst', 'concavity_worst',
         'concave points_worst', 'symmetry_worst',
         'fractal_dimension_worst']]
y = wbc[["diagnosis"]]

# Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                   random_state=123)

# Instanciamos el objeto
xg_cl = xgb.XGBClassifier(objective="binary:logistic",
                          n_estimators=10, seed=123)

# Ajustamos a los datos de entrenamiento
xg_cl.fit(X_train, y_train)

# Predecimos
preds = xg_cl.predict(X_test)

# Evaluamos
score = accuracy_score(y_test, preds)
print(score)

0.956140350877193


XGBoost es un ensamble algorithm ya que usa el resultado de muchos otros modelos para poder predecir. Usa los árboles de decisión como base learners

Boosting se puede entender como un concepto que se aplica a un set de modelos de machine learning, es un algoritmo de ensamble. Se usa para convertir un conjunto de weak learners en un strong learner (cualquier algoritmo que pueda ser perfeccionado para lograr una mejor performance)

Cross-validation en XGBoost

Para usar cross-validation directamente con xgboost debemos transformar la data en un formato especifico llamado DMatrix

In [3]:
import xgboost as xgb
import pandas as pd


# Traemos los datos
wbc = pd.read_csv("datasets/MLTreeModels/wbc.csv")
# Reemplazamos por 1 y 0
wbc['diagnosis'] = wbc['diagnosis'].replace(['M', 'B'], [1, 0])
X = wbc[['radius_mean', 'texture_mean', 'perimeter_mean',
         'area_mean', 'smoothness_mean', 'compactness_mean',
         'concavity_mean', 'concave points_mean', 'symmetry_mean',
         'fractal_dimension_mean', 'radius_se', 'texture_se',
         'perimeter_se', 'area_se', 'smoothness_se',
         'compactness_se', 'concavity_se', 'concave points_se',
         'symmetry_se', 'fractal_dimension_se', 'radius_worst',
         'texture_worst', 'perimeter_worst', 'area_worst',
         'smoothness_worst', 'compactness_worst', 'concavity_worst',
         'concave points_worst', 'symmetry_worst',
         'fractal_dimension_worst']]
y = wbc[["diagnosis"]]
churn_dmatrix = xgb.DMatrix(data=X, label=y)

params = {"objective": "binary:logistic", "max_depth":4}

cv_results = xgb.cv(dtrain=churn_dmatrix, params=params, nfold=4,
                    num_boost_round=10, metrics="error", as_pandas=True)

performance = round((1 - cv_results["test-error-mean"]).iloc[-1], 4)
print(f"Accuracy: {performance}")

Accuracy: 0.9525


XGBoost es útil cuando se tienen grandes cantidades de datos (aunque basta con el numero de caracteristicas sea menor al numero de filas)

Funciona bien cuando hay una mezcla de variables categoricas y numericas, o cuando son solo numericas

No es un buen algoritomo cuandos se trata de reconocimiento de imagenes, computer vision o NLP (para lo que es mejor usar Deep Learning)

### 2- Regresión con XGBoost

Predecir valores continuos, en la mayoría de los casos se usa RMSE como métrica para medire la performanca, aunque tambien se usa frecuentemente MAE

#### 2.1 Funciones objetivo

Miden que tan lejos estan las predicciones de los valores reales, el objetivo es encontrar el modelo que minimice el valor de la funcion objetivo

En XGBoost las funciones objetivo mas comunes son reg:linear (para problemas de regresión), reg:logistic (cuando se busca la clasificación de un registro) y binary:logistic (cuando se quiere saber la probabilidad de un suceso)

#### 2.2 Base Learners

XGBoost combina muchos modelos individuales para poder predecir, cada uno de esos modelos individuales se conoce como base learner. Lo que se busca es que estos base learner sean un poco mejor que un modelo aleatorio en predecir una parte especifica del dataset y uniformamente malos en predecir lo demás

In [6]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

names =  ["crime", "zone", "industry", "charles", "no", "rooms", "age", "distance", "radial",
          "tax", "pupil", "aam", "lower", "med_price"]
boston_data = pd.read_csv("datasets/XGBoost/housing.csv", names=names, sep='\s+')
boston_data.head()

,crime,zone,industry,charles,no,rooms,age,distance,radial,tax,pupil,aam,lower,med_price
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [7]:
X, y = boston_data.iloc[:, :-1], boston_data.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=123)

xg_reg = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=10,
                          seed=123)

xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

In [8]:
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, preds))
print(f"RMSE: {rmse}")

RMSE: 3.7824432076699046


#### 2.3 Regularization

La función objetivo no solo nos indica que tan bueno es el modelo, sino que tambien que tan complejo es. Se le dice regularizacion a la idea de penalizar los modelos mientras estos se van volviendo complejos.

La idea es obtener un modelo preciso, pero a la vez lo más simple posible

Algunos de los parametros que permite aplicar regularización en XGBoost son:

Gamma: Controla si un nodo se va a dividir o no basado en la reducción esperada de la función objetivo luego de hacer la división.  Mientras mayor es gamma, menos divisiones debiesen hacerse

Alpha: L1 Regularization, penaliza el peso de las hojas. Mientras mayor el valor, mayor sera la regularizacion y el peso de las hojas tenderá a cero

Lambda: L2 Regularization. Es una penalización un poco más sueve que la de L1. LLevan a los pesos de las hojas a descender suavemente


In [9]:
import xgboost as xgb
import pandas as pd

names =  ["crime", "zone", "industry", "charles", "no", "rooms", "age", "distance", "radial",
          "tax", "pupil", "aam", "lower", "med_price"]
boston_data = pd.read_csv("datasets/XGBoost/housing.csv", names=names, sep='\s+')
X, y = boston_data.iloc[:, :-1], boston_data.iloc[:,-1]
boston_dmatrix = xgb.DMatrix(data=X, label=y)
params = {"objetive": "reg:linear", "max_depth":4, "verbosity":0}

l1_params = [1, 10, 100]
rmses_l1 = []

for reg in l1_params:
    params["alpha"] = reg
    cv_results = xgb.cv(dtrain=boston_dmatrix, params=params, nfold=4,
                        num_boost_round=10, metrics="rmse", as_pandas=True,
                        seed=123)
    rmses_l1.append(cv_results["test-rmse-mean"].tail(1).values[0])
    
print("Best rmse as a function of l1:")
print(pd.DataFrame(list(zip(l1_params, rmses_l1)), columns=["l1", "rmse"]))

Best rmse as a function of l1:
    l1      rmse
0    1  3.461474
1   10  3.821152
2  100  4.645518


### 3- Fine-Tunining

Veamos como sería un entrenamiento sin optimizar los parametros

In [10]:
import xgboost as xgb
import pandas as pd
import numpy as np

# Load data
names =  ["crime", "zone", "industry", "charles", "no", "rooms", "age", "distance", "radial",
          "tax", "pupil", "aam", "lower", "med_price"]
boston_data = pd.read_csv("datasets/XGBoost/housing.csv", names=names, sep='\s+')
X, y = boston_data.iloc[:, :-1], boston_data.iloc[:,-1]
boston_dmatrix = xgb.DMatrix(data=X, label=y)
params = {"objetive": "reg:linear", "verbosity":0}

# cross validation
cv_results = xgb.cv(dtrain=boston_dmatrix, params=params, nfold=4,
                    metrics="rmse", as_pandas=True,
                    seed=123)

# Evaluacion
rmse_test = cv_results["test-rmse-mean"].tail(1).values[0]
print(f"El RMSE del test es {rmse_test}")

El RMSE del test es 3.43206225


Ahora probamos con un modelo modificando algunos parametros

In [12]:
import xgboost as xgb
import pandas as pd
import numpy as np

# Load data
names =  ["crime", "zone", "industry", "charles", "no", "rooms", "age", "distance", "radial",
          "tax", "pupil", "aam", "lower", "med_price"]
boston_data = pd.read_csv("datasets/XGBoost/housing.csv", names=names, sep='\s+')
X, y = boston_data.iloc[:, :-1], boston_data.iloc[:,-1]
boston_dmatrix = xgb.DMatrix(data=X, label=y)
params = {"objective": "reg:linear", "verbosity":0, "colsample_bytree": 0.3,
          "learning_rate": 0.03, "max_depth": 4}

# cross validation
cv_results = xgb.cv(dtrain=boston_dmatrix, params=params, nfold=4,
                    metrics="rmse", as_pandas=True, num_boost_round=200,
                    seed=123)

# Evaluacion
rmse_test = cv_results["test-rmse-mean"].tail(1).values[0]
print(f"El RMSE del test es {rmse_test}")

El RMSE del test es 3.4437465


Iteramos sobre una lista de posibilidades para encontrar el mejor rmse

In [13]:
import xgboost as xgb
import pandas as pd
import numpy as np

# Load data
names =  ["crime", "zone", "industry", "charles", "no", "rooms", "age", "distance", "radial",
          "tax", "pupil", "aam", "lower", "med_price"]
boston_data = pd.read_csv("datasets/XGBoost/housing.csv", names=names, sep='\s+')
X, y = boston_data.iloc[:, :-1], boston_data.iloc[:,-1]
boston_dmatrix = xgb.DMatrix(data=X, label=y)
params = {"objective": "reg:linear", "verbosity":0, "colsample_bytree": 0.3,
          "learning_rate": 0.03, "max_depth": 4}

# Definimos distintas posibilidades de num_boost_round
num_rounds = [5, 10, 15, 17, 19, 20]

# Iteramos por cada posible combinacion
final_rmse_per_round = []
for n_rounds in num_rounds:
    cv_results = xgb.cv(dtrain=boston_dmatrix, params=params, nfold=3,
                        num_boost_round=n_rounds, metrics="rmse",
                        as_pandas=True, seed=123)
    # Agregamos a la lista de resultados
    final_rmse_per_round.append(cv_results["test-rmse-mean"].tail().values[-1])
    
# Unimos los resultados
num_rounds_rmses = pd.DataFrame(list(zip(num_rounds, final_rmse_per_round)),
                                columns=["num_boost_round", "rmse"])
print(num_rounds_rmses)

   num_boost_round       rmse
0                5  20.851624
1               10  18.234990
2               15  16.062435
3               17  15.281037
4               19  14.535187
5               20  14.183934


#### 3.1 Tunable parameters

Para modelos basados en arboles se pueden ajustar los siguientes parametros

    - learning rate: Que tan rápido se ajusta el modelo al error residual
    - gamma: Cual es el minimo de ganacia en la funcion objetivo para que un nodo se divida
    - lambda: Regularización L2
    - alpha: Regularización L1
    - max_depth: que tanto puede crecer un arbol en las rondas de boosting
    - subsample: (va entre 0 y 1) El porcentaje de datos que se utilizará para entrenar el modelo en cada ronda de Boosting
    - colsample_bytree: Fracción de caracteristicas que se pueden usar en cada ronda de boosting (es un valor entre 0 y 1)

Para modelos basados en regresiones lineales se puede modificar

    - lambda: Regularización L2
    - alpha: Regularización L1
    - lambda_bias: L2 en bias

Para ambos es posible modificar el numero de estimadores que se usan

#### 3.2 GridSearch y RandomSearch

Como escogemos la mejor combinación de parametros cuando estos interactuan en relaciones no lineales

Grid Search es un método que busca exhaustivamente sobre un set de hiperparametros, esto quiere decir, que el numero de modelos calculados será igual a la suma de los productos de el numero de hiperparametros a optimizar por la cantidad de opciones en cada uno de estos

In [14]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV

# Load data
names =  ["crime", "zone", "industry", "charles", "no", "rooms", "age", "distance", "radial",
          "tax", "pupil", "aam", "lower", "med_price"]
boston_data = pd.read_csv("datasets/XGBoost/housing.csv", names=names, sep='\s+')
X, y = boston_data.iloc[:, :-1], boston_data.iloc[:,-1]
boston_dmatrix = xgb.DMatrix(data=X, label=y)

# Parametros a tunnear
gbm_param_grid = {"learning_rate": [0.01, 0.1, 0.5, 0.9],
                  "n_estimators": [200], "subsample": [0.3, 0.5, 0.9]}

gbm = xgb. XGBRegressor()
grid_mse = GridSearchCV(estimator=gbm, param_grid=gbm_param_grid,
                        scoring="neg_mean_squared_error", cv=4, verbose=0)
grid_mse.fit(X, y)

best_params = grid_mse.best_params_
best_score = grid_mse.best_score_

print(f"Mejores parametros {best_params}")
print(f"Mejor score {best_score}")

Mejores parametros {'learning_rate': 0.1, 'n_estimators': 200, 'subsample': 0.9}
Mejor score -20.346251013052715


Random Search permite crear un conjunto infinito de hiperparametros y luego se definen el numero de iteraciones en las que la busqueda del modelo optimo continua

Para cada una de las iteraciones se selecciona un valor random de los hiperparametros y se recalcula el modelo con eso

Cuando se alcanza el máximo de iteraciones antes fijado, simplemente se escoge la que tuvo el mejor resultado

In [15]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

# Load data
names =  ["crime", "zone", "industry", "charles", "no", "rooms", "age", "distance", "radial",
          "tax", "pupil", "aam", "lower", "med_price"]
boston_data = pd.read_csv("datasets/XGBoost/housing.csv", names=names, sep='\s+')
X, y = boston_data.iloc[:, :-1], boston_data.iloc[:,-1]
boston_dmatrix = xgb.DMatrix(data=X, label=y)

# Parametros a tunnear
gbm_param_grid = {"learning_rate": np.arange(0.01, 0.9, .05),
                  "n_estimators": [200], "subsample": np.arange(0.3, 0.9, .05)}

gbm = xgb. XGBRegressor()
grid_mse = RandomizedSearchCV(estimator=gbm, param_distributions=gbm_param_grid,
                              scoring="neg_mean_squared_error", cv=4, verbose=0,
                              n_iter=25)
grid_mse.fit(X, y)

best_params = grid_mse.best_params_
best_score = grid_mse.best_score_

print(f"Mejores parametros {best_params}")
print(f"Mejor score {best_score}")

Mejores parametros {'subsample': 0.44999999999999996, 'n_estimators': 200, 'learning_rate': 0.060000000000000005}
Mejor score -18.867646521373217


### 4- Pipelines

XGBoost puede ser incorporado en una pipeline de scikit-learn

Las pipelines reciben una lista de tuplas como input, en donde el primer elemento de cda tupla es el en nombre del paso y luego el segundo elemento es el objeto con el que se realiza la operación (debe ser compatible con scikit-learn)

In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

names =  ["crime", "zone", "industry", "charles", "no", "rooms", "age", "distance", "radial",
          "tax", "pupil", "aam", "lower", "med_price"]
boston_data = pd.read_csv("datasets/XGBoost/housing.csv", names=names, sep='\s+')
X, y = boston_data.iloc[:, :-1], boston_data.iloc[:,-1]

rf_pipeline = Pipeline([("st_scaler", StandardScaler()),
                        ("rf_model", RandomForestRegressor())])

scores = cross_val_score(rf_pipeline, X, y, scoring="neg_mean_squared_error", cv=10)

final_avg_rsme = np.mean(np.sqrt(np.abs(scores)))

print(f"RMSE final: {final_avg_rsme}")

RMSE final: 4.17692044815858


#### 4.1- LabelEncoder

Convierte columnas categoricas en enteros

In [26]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("datasets/XGBoost/ames_unprocessed_data.csv", sep=',')

# Rellenamos los valores vacios
df.LotFrontage = df.LotFrontage.fillna(0)

# Buscamos las columnas que son categoricas
categorical_mask = (df.dtypes == object)

# Una vez que sabemos cuales son categoricas y cuales no, creamos una lista solo con categoricas
categorical_columns = df.columns[categorical_mask].tolist()

# Creamos el objeto de LabelEncoder
le = LabelEncoder()

# Aplicamos LabelEncoder en cada una de las columnas categoricas
df[categorical_columns] = df[categorical_columns].apply(lambda x: le.fit_transform(x))


#### 4.2- OneHotEncoder

Convierte una columna categorica en multiples columnas binarias

In [37]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

df = pd.read_csv("datasets/XGBoost/ames_unprocessed_data.csv", sep=',')

# Rellenamos los valores vacios
df.LotFrontage = df.LotFrontage.fillna(0)

# Buscamos las columnas que son categoricas
categorical_mask = (df.dtypes == object)

# Una vez que sabemos cuales son categoricas y cuales no, creamos una lista solo con categoricas
categorical_columns = df.columns[categorical_mask].tolist()

# Creamos el objeto de OneHotEncoder
ohe = OneHotEncoder(sparse=False)

# Usamos ColumnTransformer para especificar las columnas en que queremos aplicarlo
column_trans = ColumnTransformer([("onehot_categorical", ohe, categorical_columns)],
                                 remainder="passthrough")
df_encoded = column_trans.fit_transform(df)

print("Forma del dataframe original", df.shape)

# Print the shape of the transformed array
print("Forma del array resultante", df_encoded.shape)

Forma del dataframe original (1460, 21)
Forma del array resultante (1460, 62)


#### 4.3- DictVectorizer

Ni LabelEncoder ni OneHotEncoder se pueden utilizar en una pipeline, sin embargo, DictVectorizer nos permite hacer lo mismo en una sola linea y se puede incorporar a una Pipeline

Es usualmente usado en procesamiento de texto. Busca convertir listas de caracteristicas en vectores

Lo que necesitamos es convertir edataframe de pandas en una lista de dcicionarios

In [39]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer

df = pd.read_csv("datasets/XGBoost/ames_unprocessed_data.csv", sep=',')

# Rellenamos los valores vacios
df.LotFrontage = df.LotFrontage.fillna(0)

# Convertimos el dataframe a diccionario
df_dict = df.to_dict(orient='records')

# Creamos el objeto DictVectorizer
dv = DictVectorizer(sparse=False)

# Aplicamos la tranformación
df_encoded = dv.fit_transform(df_dict)

# Imprimimos el resultado para las 5 primeras filas
print(df_encoded[:5,:])

# Imprimimos el resultado generado para el vocabulario
print(dv.vocabulary_)

[[3.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 0.000e+00 2.000e+00 5.480e+02 1.710e+03 1.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00
  8.450e+03 6.500e+01 6.000e+01 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 5.000e+00 7.000e+00
  0.000e+00 0.000e+00 1.000e+00 0.000e+00 2.085e+05 2.003e+03]
 [3.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  1.000e+00 1.000e+00 2.000e+00 4.600e+02 1.262e+03 0.000e+00 0.000e+00
  0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  9.600e+03 8.000e+01 2.000e+01 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 

#### 4.4 Full Pipeline

In [40]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

df = pd.read_csv("datasets/XGBoost/ames_unprocessed_data.csv", sep=',')

X, y = df.iloc[:, :-1], df.iloc[:,-1]

# Fill LotFrontage missing values with 0
X.LotFrontage = X.LotFrontage.fillna(0)

# Setup the pipeline steps: steps
steps = [("ohe_onestep", DictVectorizer(sparse=False)),
         ("xgb_model", xgb.XGBRegressor())]

# Create the pipeline: xgb_pipeline
xgb_pipeline = Pipeline(steps)

# Fit the pipeline
xgb_pipeline.fit(X.to_dict('records'), y)

Pipeline(steps=[('ohe_onestep', DictVectorizer(sparse=False)),
                ('xgb_model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=6, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=0, num_parallel_tree=1, random_state=0,
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                              subsample=1, tree_method='exact',
                              validate_parameters=1, verbosity=None))])

#### 4.5- Componentes adicionales para Pipelines

sklearn_pandas: Intenta unir scikit-learn con pandas para aquellos casos en que no funcionan bien juntas. 
- Tiene una clase llamada DataFrameMapper que permite convertir rapidamente de dataframe a el formato que scikit-learn necesita 
- CategoricalImputer permite imputar valores a las variables categoricas antes de que se conviertan en enteros

sklearn.preprossing.Imputer: Imputar valores a columnas numericas

sklearn.pipeline.FeatureUnion: Permite combinar multiples pipelines de caracteristicas en solo una

In [77]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

df = pd.read_csv("datasets/XGBoost/ames_unprocessed_data.csv", sep=',')

X, y = df.iloc[:, :-1], df.iloc[:,-1]

X.LotFrontage = X.LotFrontage.fillna(0)

steps = [("ohe_onestep", DictVectorizer(sparse=False)),
         ("xgb_model", xgb.XGBRegressor(max_depth=2, objective="reg:linear"))]

xgb_pipeline = Pipeline(steps)

cross_val_scores = cross_val_score(xgb_pipeline, X.to_dict('records'), y,
                                   scoring="neg_mean_squared_error", cv=10)

print("10-fold RMSE: ", np.mean(np.sqrt(np.abs(cross_val_scores))))

10-fold RMSE:  27683.04157118635
